<a href="https://colab.research.google.com/github/davidlu51412/COMP562FinalProj/blob/main/V1_1_Class1_COMP_562_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import numpy as np
import tensorflow as tf

In [ ]:
import os
os.mkdir("generators")
os.mkdir("discriminators")
os.mkdir("train_plots")

# Create Models

In [ ]:
# WE could seperate the class and gan discriminator into two different things,
# where they both recieve the generated output individually

#  WE CAN PROBABLY PUT DROPOUT LAYERS TO HELP DECREASE OVERTRAINING
learning_step = .0001

def create_class_discriminator(image_shape, class_dim):
  # get convolved image from (base) and try to classify  what class this image is
  disc_input = tf.keras.Input(shape=(image_shape))
  disc_x = tf.keras.layers.Conv2D(64, (3,3),
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_input)
  disc_x = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), 
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_x)
  disc_x = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), 
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_x)
  disc_x = tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), 
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_x)
  disc_x = tf.keras.layers.Flatten()(disc_x)
  disc_x = tf.keras.layers.Dropout(.4)(disc_x)
  disc_x = tf.keras.layers.Dense(class_dim, activation="sigmoid")(disc_x)
  class_discriminator = tf.keras.Model(inputs=disc_input, outputs=disc_x)
  tf.keras.utils.plot_model(class_discriminator, "class_discriminator.png", show_shapes=True)
  optimizerfn = tf.keras.optimizers.Adam(learning_rate=learning_step, beta_1=.9)
  class_discriminator.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                        optimizer=optimizerfn, metrics=['accuracy'])
  return class_discriminator

def create_gan_discriminator(image_shape):
  # gets convolved image from (base) and try to classify if this is a real or fake image
  disc_input = tf.keras.Input(shape=image_shape)
  disc_x = tf.keras.layers.Conv2D(64, (3,3),
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_input)
  disc_x = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), 
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_x)
  disc_x = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), 
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_x)
  disc_x = tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), 
                                  activation=tf.keras.layers.LeakyReLU(.2),
                                  padding='same')(disc_x)
  disc_x = tf.keras.layers.Flatten()(disc_x)
  disc_x = tf.keras.layers.Dropout(.4)(disc_x)
  disc_x = tf.keras.layers.Dense(1, activation='sigmoid')(disc_x)
  full_discriminator = tf.keras.Model(inputs=disc_input, outputs=disc_x)
  tf.keras.utils.plot_model(full_discriminator, "gan_discriminator.png", show_shapes=True)
  optimizerfn = tf.keras.optimizers.Adam(learning_rate=learning_step, beta_1=.9)
  full_discriminator.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
                        optimizer=optimizerfn, metrics=['accuracy'])
  return full_discriminator

def create_generator(input_dim):
  # maybe we can input the generated image into itself
  # note: we will pass random noise to this generator
  # (in hopes it will generate the correct variations of an object)
  # two inputs: random noise and the given image class we want to generate
  gen_x_input = tf.keras.Input(shape=(input_dim))
  # 4x4 upscaling foundation
  num_nodes = 4*4*256 
  gen_x = tf.keras.layers.Dense(num_nodes)(gen_x_input)
  gen_x = tf.keras.layers.Reshape((4, 4, 256))(gen_x)
  # upsize
  # 8x8
  gen_x = tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2),  
                                          activation=tf.keras.layers.LeakyReLU(.2),
                                          padding="same")(gen_x)

  # 16x16
  gen_x = tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2),  
                                          activation=tf.keras.layers.LeakyReLU(.2),
                                          padding="same")(gen_x)
  # 32x32
  gen_x = tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2),  
                                          activation=tf.keras.layers.LeakyReLU(.2),
                                          padding="same")(gen_x)
  # 28x28
  gen_x = tf.keras.layers.Conv2D(1, (5,5), activation='tanh')(gen_x)
  generator = tf.keras.Model(inputs=gen_x_input, outputs=gen_x)
  tf.keras.utils.plot_model(generator, "generator.png", show_shapes=True)
  return generator


def create_gan(generator, gan_discriminator, input_dim):
  # will be same input as generator
  gan_x_input = tf.keras.Input(shape=(input_dim))
  gan_x = generator(gan_x_input) 

  # we want to train the gan to match the classifier and if it's real or fake
  real_or_fake = gan_discriminator(gan_x)
  gan_discriminator.trainable = False

  gan = tf.keras.Model(inputs=gan_x_input, outputs=real_or_fake)
  tf.keras.utils.plot_model(gan, "gan.png", show_shapes=True)
  optimizerfn = tf.keras.optimizers.Adam(learning_rate=learning_step, beta_1=.9)
  gan.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
                        optimizer=optimizerfn, metrics=['accuracy'])
  
  return gan

def create_class_discriminator_features(generator, class_discriminator, generator_input_dim):
  c_d_features_input = tf.keras.Input(shape=(generator_input_dim))
  c_d_features = generator(c_d_features_input)
  c_d_features = class_discriminator(c_d_features)
  class_discriminator.trainable = False
  c_d_features_model = tf.keras.Model(inputs=c_d_features_input, outputs=c_d_features)
  tf.keras.utils.plot_model(c_d_features_model, "c_d_feature.png", show_shapes=True)
  optimizerfn = tf.keras.optimizers.Adam(learning_rate=learning_step, beta_1=.9)
  c_d_features_model.compile(loss=tf.keras.losses.MeanSquaredError(), 
                        optimizer=optimizerfn, metrics=['accuracy'])
  return c_d_features_model


# Training Functions

In [ ]:
# important functions for training
from matplotlib import pyplot
import shutil

def generate_class_disc_samples(dataset, num_samples, labels):
  rand_idx = np.random.randint(0, dataset.shape[0], num_samples)
  real_im = dataset[rand_idx]
  # real class label of "1"
  real_label = labels[rand_idx]
  return real_im, real_label

def generate_real_samples(dataset, num_samples, labels, class_num):
  
  class_data = dataset[labels==class_num]
  # get random "real" samples
  rand_idx = np.random.randint(0, class_data.shape[0], num_samples)
  real_im = class_data[rand_idx]
  # real class label of "1"
  true_class = np.ones((num_samples, 1))  
  return real_im, true_class

def generate_fake_samples(generator, input_dim, num_samples):
  # generate noise input for GAN with randomness and it's own prediction
  inputs = np.random.randn(input_dim*num_samples)
  # reshape into dimensions for the network
  inputs = inputs.reshape(num_samples, input_dim)
  fake_im = generator.predict(inputs)
  true_class = np.zeros((num_samples, 1))
  return fake_im, true_class

def generate_GAN_samples(generator, input_dim, num_samples):
  # generate noise input for GAN with randomness and it's own prediction
  inputs = np.random.randn(input_dim*num_samples)
  # reshape into dimensions for the network
  inputs = inputs.reshape(num_samples, input_dim)
  true_class = np.ones((num_samples, 1))
  return inputs, true_class

# eval current performance
def summarize_performance(epoch, generator, gan_discriminator, dataset, input_dim, labels, class_names, class_num,
                          num_samples=150):
  # prepare real samples
  X_real, real_gan = generate_real_samples(dataset, num_samples, labels, class_num)
  # evaluate discriminator on real examples
  real_eval = gan_discriminator.evaluate(X_real, real_gan, verbose=0)

  # prepare fake examples
  x_fake, gan_fake = generate_fake_samples(generator, input_dim, num_samples)
  # evaluate discriminator on fake examples
  fake_eval = gan_discriminator.evaluate(x_fake, gan_fake, verbose=0)
  
  # summarize discriminator performance
  print("____________________")
  print("Epoch:",epoch+1, "|| Class:", class_names[class_num])
  print(">real_eval:", real_eval)
  print(">fake_eval:", fake_eval)
  print("____________________")
  # save plot
  save_plot(x_fake, epoch, class_names, class_num, 4)
  # save the generator model tile file
  filename = 'generator_model_' + str(epoch+1) + '_' + class_names[class_num] + '.h5' 
  generator.save(filename)
  shutil.move(filename, './generators/'+filename)

  filename = 'gan_discriminator_model_' + str(epoch+1) + '_' + class_names[class_num] + '.h5' 
  gan_discriminator.save(filename)
  shutil.move(filename, './discriminators/'+filename)

# create and save a plot of generated images
def save_plot(examples, epoch, class_names, class_num, n=4):
  # scale from [-1,1] to [0,1]
  examples = (examples + 1) / 2.0
  # plot images
  for i in range(n * n):
    # define subplot
    pyplot.subplot(n, n, 1 + i)
    pyplot.title(class_names[class_num])
    pyplot.tight_layout(pad=.5)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(examples[i,:,:,0], cmap="gray")

	# save plot to file
  filename = 'generated_plot_e' + str(epoch+1) + '_' + class_names[class_num] + '.png'
  pyplot.savefig(filename)
  pyplot.close()
  shutil.move(filename, './train_plots/'+filename)
  

def show_rand_plot(generators, input_dim, num_samples, labels, class_names, n=4):
  # prepare fake examples
  x_fake, true_class = generate_fake_samples(generator, input_dim, num_samples, labels)
  # scale from [-1,1] to [0,1]
  x_fake = (x_fake + 1) / 2.0
  # plot images
  for i in range(n * n):
    # define subplot
    pyplot.subplot(n, n, 1 + i)
    pyplot.title(class_names[np.where(real_label[i] == 1)[0][0]])
    pyplot.tight_layout(pad=.5)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(x_fake[i,:,:,0], cmap="gray")
# train the generator and discriminator
def train(generators, gan_discriminators, gans, class_discriminator, cd_features,
          dataset, labels, input_dim, sample_num, n_epochs=200, n_batch=128):
          
  batches_per_epoch = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_epochs):
    # batch training (for 1 epoch)
    for j in range(batches_per_epoch):
      d_loss_gan_disc_real = 0
      d_loss_gan_disc_fake = 0
      g_loss = 0
      cd_feature_loss = 0
      real_im, real_label = generate_class_disc_samples(dataset, half_batch, labels) # train class disc
      class_discriminator.train_on_batch(real_im, real_label)
      
      
      for k in range(len(generators)):
        # get this classes generators and gan
        gan_discriminator = gan_discriminators[k]
        generator = generators[k]
        gan = gans[k]
        cd_feature = cd_features[k]

        real_im, _ = generate_real_samples(dataset, half_batch, labels, k)
        class_disc_features = class_discriminator.predict(real_im) # get this classes' expected feature from class disc
        cd_input, _ = generate_GAN_samples(generator, input_dim, half_batch) # get noise for cd_feature input
        cd_feature_loss_local, _ = cd_feature.train_on_batch(cd_input, class_disc_features)
        cd_feature_loss += cd_feature_loss_local
        
        real_im, real_gan = generate_real_samples(dataset, half_batch, labels, k)
        d_loss_real_local, _ = gan_discriminator.train_on_batch(real_im, real_gan) # gan loss real
        d_loss_gan_disc_real += d_loss_real_local
        fake_im, fake_gan = generate_fake_samples(generator, input_dim, half_batch)
        d_loss_fake_local, _ = gan_discriminator.train_on_batch(fake_im, fake_gan) # gan loss fake
        d_loss_gan_disc_fake += d_loss_fake_local

        gan_input, inverted_label = generate_GAN_samples(generator, input_dim, half_batch) # returns with random noise for gan input
        g_loss_local, _ = gan.train_on_batch(gan_input, inverted_label) # train generator with inverted labels via gan
        g_loss += g_loss_local
      # summarize loss on this batch
      if (j+1) % n_batch == 0:
        print("Epoch:", i+1, " - Batch:",str(j+1)+"/"+str(batches_per_epoch),
              "avg_d_loss_gan (real/fake):" + str(d_loss_gan_disc_real/len(labels)) + "," + str(d_loss_gan_disc_fake/len(labels)),
              "avg_gan_loss:" + str(g_loss/len(labels)),"cd_feature_loss:" + str(cd_feature_loss/len(labels)))
    # evaluate the model performance, sometimes
    if (i+1) % 10 == 0:
      for k in range(len(generators)):
        summarize_performance(i, generators[k], gan_discriminators[k], dataset, input_dim, labels, class_names, k, sample_num)

# Init and Train

In [ ]:
# initializations before training
# load dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
class_names = ['zero', 'one', 'two', 'three', 'four',
               'five', 'six', 'seven', 'eight', 'nine']
# for tensorflow format (they assume 3rd dimension with color channels)
train_images = np.expand_dims(train_images, 3)
train_images = np.interp(train_images, (train_images.min(), train_images.max()), (-1, +1))

test_images = np.expand_dims(test_images, 3)
test_images = np.interp(test_images, (test_images.min(), test_images.max()), (-1, +1))

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
im_shape = train_images[0].shape  # is the noise dim
sample_num = 16 # make this a full square
# length of noise vector for random generation
label_shape = len(class_names)
generator_input_dim = 100
# init the generator array
generators = []
gans = []
gan_discriminators = []
class_discriminator = create_class_discriminator(im_shape, label_shape,)
cd_features = []

# create the generator array
for i in range(label_shape):
  generators.append(create_generator(generator_input_dim))
  gan_discriminators.append(create_gan_discriminator(im_shape))
  gans.append(create_gan(generators[i], gan_discriminators[i], generator_input_dim))
  cd_features.append(create_class_discriminator_features(generators[i], class_discriminator, generator_input_dim))

In [ ]:
train(generators, gan_discriminators, gans, class_discriminator,  cd_features,
      train_images, train_labels, generator_input_dim, sample_num, n_epochs=1000, 
      n_batch=1024)

____________________
Epoch: 10 || Class: zero
>real_eval: [0.357504665851593, 0.9375]
>fake_eval: [0.47031107544898987, 0.875]
____________________
____________________
Epoch: 10 || Class: one
>real_eval: [0.6140294671058655, 0.75]
>fake_eval: [0.6264340877532959, 0.6875]
____________________
____________________
Epoch: 10 || Class: two
>real_eval: [0.7770315408706665, 0.625]
>fake_eval: [0.3382793664932251, 0.875]
____________________
____________________
Epoch: 10 || Class: three
>real_eval: [0.3828827440738678, 0.8125]
>fake_eval: [0.33611831068992615, 0.9375]
____________________
____________________
Epoch: 10 || Class: four
>real_eval: [0.06815672665834427, 1.0]
>fake_eval: [0.0802386924624443, 1.0]
____________________
____________________
Epoch: 10 || Class: five
>real_eval: [0.22110377252101898, 0.875]
>fake_eval: [0.14747223258018494, 1.0]
____________________
____________________
Epoch: 10 || Class: six
>real_eval: [0.3401579260826111, 0.9375]
>fake_eval: [0.40418776869773865

KeyboardInterrupt: ignored

In [ ]:
show_rand_plot(generator, im_shape, sample_num, train_labels, class_names, 4)

In [ ]:
import time
for i in range(12):
  time.sleep(60*60)